In [ ]:
matrix = os.path.join(os.path.dirname(profBCorpusPath), config.FEATURES_MATRIX)

In [ ]:
def randomForest_gridsearch(dataFile, modelTarget, upsample=False):
    logger.info("randomForest_gridsearch(dataFile={df}, modelTarget={mt})".format(df=dataFile, mt=modelTarget))

    from sklearn.model_selection import train_test_split

    samples = pd.read_excel(dataFile)
    samples = samples.rename(index=str,
                             columns={"Presence Class": "PresenceClass", "Co-presence Class": "CopresenceClass"})

    names = ("Expert", "Head_Entropy_Start", "Head_Entropy_Mid", "Head_Entropy_End", "Avg_HandEntropy_Begin",
             "Avg_HandEntropy_Mid", "Avg_HandEntropy_End", "Avg_SentenceLength_Begin", "Avg_SentenceLength_Mid",
             "Avg_SentenceLength_End", "Avg_IPUlen_Begin", "Avg_IPUlen_Middle", "Avg_IPUlen_End", "Ratio1_Begin",
             "Ratio1_Mid", "Ratio1_End", "Ratio2_Begin", "Ratio2_Mid", "Ratio2_End", "Duration")

    samples_split = []
    if (modelTarget == "presence"):
        samples_split.append(samples[samples.PresenceClass == 1])
        samples_split.append(samples[samples.PresenceClass == 2])
        samples_split.append(samples[samples.PresenceClass == 3])

    elif (modelTarget == "copresence"):
        samples_split.append(samples[samples.CopresenceClass == 1])
        samples_split.append(samples[samples.CopresenceClass == 2])
        samples_split.append(samples[samples.CopresenceClass == 3])
    else:
        sys.exit("Invalid input. Please pick between presence and copresence")

    maxClassSize = max(samples_split[0].shape[0], samples_split[1].shape[0], samples_split[2].shape[0])

    if upsample:
        upsampled = []
        # todo upsample with SMOTE algorithm ? https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.over_sampling.SMOTE.html
        for idx, samples in enumerate(samples_split):
            if (samples.shape[0] == maxClassSize):
                upsampled.append(samples)
            else:
                logger.debug("resample: adding " + str(maxClassSize - samples.shape[0]) + " samples to class " + str(
                    idx + 1) + " to reach " + str(maxClassSize))
                upsampled.append(resample(samples, replace=True, n_samples=maxClassSize, random_state=None))

        balanced_set = pd.concat(upsampled)
        X = np.nan_to_num(balanced_set.as_matrix(names))

        if (modelTarget == "presence"):
            y = np.array(balanced_set["PresenceClass"].tolist())

        else:
            y = np.array(balanced_set["CopresenceClass"].tolist())

    else:

        X = np.nan_to_num(samples[list(names)])
        if modelTarget == "presence":
            y = samples.PresenceClass
        else:
            y = samples.CopresenceClass

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    logger.debug("X_train ", X_train.shape, "X_test", X_test.shape, "y_train", y_train.shape, "y_test", y_test.shape)

    forest = RandomForestClassifier()

    # print X.shape
    print(modelTarget + " random forest")
    from sklearn.model_selection import GridSearchCV  # todo move to beginning with other imports

    # n_estimators = np.concatenate((np.arange(1,10), np.arange(10,100,10)))
    n_estimators = np.arange(1, 100)
    class_weights = [None, 'balanced', 'balanced_subsample']
    folds = np.concatenate((np.arange(1, 10), np.arange(10, len(X), 10)))
    param_grid = dict(n_estimators=n_estimators, class_weight=class_weights)
    grid = GridSearchCV(estimator=forest, param_grid=param_grid,
                        scoring=['f1_macro', 'precision_macro', 'recall_macro'],
                        refit='precision_macro',
                        cv=20,
                        return_train_score=True,
                        verbose=1)

    grid = grid.fit(X_train, y_train)

    print("TEST Score : ", grid.score(X_test, y_test))

    results = grid.cv_results_
    # print("best params ", grid.best_params_)
    # print("best score ", grid.best_score_)

    return grid